In [21]:
import numpy as np 


def load_channel_data(filename):
    """
    Load in channel flow data.
    """

    # Load in data from txt file
    data = np.loadtxt(filename)
    y = data[:, 0]
    k = data[:, 1]
    eps = data[:, 2]
    grad_u_flat = data[:, 3:12]
    stresses_flat = data[:, 12:21]
    u = data[:, 21:]

    # Reshape grad_u and stresses to num_points X 3 X 3 arrays
    num_points = data.shape[0]
    grad_u = grad_u_flat.reshape(num_points, 3, 3)
    stresses = stresses_flat.reshape(num_points, 3, 3)

    return k, eps, grad_u, stresses, u, y


def save_channel_data(filename, y, k, epsilon, grad_u, stresses, u):
    """
    Save channel flow data.
    """
    header = 'y+, tke, epsilon, grad_u_11, grad_u_12, grad_u_13, grad_u_21, grad_u_22, grad_u_23, \
grad_u_31, grad_u_32, grad_u_33, uu_11, uu_12, uu_13, uu_21, uu_22, uu_23, uu_31, uu_32, uu_33, u_1, u_2, u_3'
    
    Ny = y.shape[0]
    output_array = np.zeros((Ny, 24))
    output_array[:, 0] = y
    output_array[:, 1] = k
    output_array[:, 2] = epsilon
    output_array[:, 3:12] = grad_u.reshape(Ny, 9)
    output_array[:, 12:21] = stresses.reshape(Ny, 9)
    output_array[:, 21:] = u
    np.savetxt(filename, output_array, header=header)


def moving_average(array, window_size):
    """
    Average over n=window_size neighboring points along axis 0. 
    """
    def _moving_average_1d(array): 
        window = np.ones(int(window_size))/float(window_size)
        new_array = np.convolve(array, window, mode='same')
        for i in range(int(window_size) // 2):
            fac = window_size / (window_size + i - window_size // 2)
            new_array[i] *= fac
            new_array[-i-1] *= fac 
        return new_array
    
    return np.apply_along_axis(_moving_average_1d, axis=0, arr=array)


def subsampling(array, interval, start=0):
    """
    Subsample an array along axis 0.   
    """
    
    return array[start::interval]

In [5]:
arr = np.arange(0, 10).reshape(5,2)
arr

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7],
       [8, 9]])

In [4]:
moving_average(arr, 3)

array([[1., 2.],
       [2., 3.],
       [4., 5.],
       [6., 7.],
       [7., 8.]])

In [23]:
subsampling(arr, 3, 1)

array([[2, 3],
       [8, 9]])

In [29]:
fname = '../Data/LM_Channel_Retau180.txt'
cg_fname = '../Data/LM_Channel_Retau180_cg2.txt'

# Load data 
tke, eps, grad_u, stresses, u, y = load_channel_data(fname)

# Coarse-grain data
window_size = 3

cg_tke = moving_average(tke, window_size=window_size)
cg_eps = moving_average(eps, window_size=window_size)
cg_grad_u = moving_average(grad_u, window_size=window_size)
cg_u = moving_average(u, window_size=window_size)

cg_tke = subsampling(cg_tke, window_size) 
cg_eps = subsampling(cg_eps, window_size) 
cg_grad_u = subsampling(cg_grad_u, window_size) 
cg_u = subsampling(cg_u, window_size)
y = subsampling(y, window_size)
stresses = subsampling(stresses, window_size)

# Save to txt file 
save_channel_data(cg_fname, y, cg_tke, cg_eps, cg_grad_u, stresses, cg_u)